In [9]:
import import_images as i
import classification_class as classification
import pandas as pd
import SIFT as SIFT


=========1. Extract SIFT features from all images=========
=========2. Stack all descriptors for clustering (BoVW) =========
=========3. Cluster descriptors using KMeans (BoVW approach) =========
========= 4. Create feature histograms for each image =========


In [10]:
metrics_results_best_methods = {
        "accuracy" : {},
        "f1-score" : {},
        "recall" : {},
        "precision" : {},
        "roc_auc" : {}
        }


name_best_models = {}

In [11]:
print("========= 5. Classification =========")
df_features = pd.DataFrame(SIFT.feature_vectors)
df_Y = pd.DataFrame(i.Y)

data_SIFT = classification.DataProcessing(df_features, df_Y, stratified=False)
env_SIFT = classification.BinaryClassification(data_SIFT, average="macro")
metrics_results = env_SIFT.CrossValidationKFold()
labels = list(metrics_results['f1-score'].keys())
results_train_KFold, results_test_KFold = env_SIFT.createMeansDataframe(metrics_results, labels)
results_train_KFold.style.highlight_max(axis=0)

best_method_name = env_SIFT.get_best_method(results_test_KFold, "F1-score", ens="Test")
print(f"Best method name: {best_method_name}")
name_best_models["SIFT + BoW"] = best_method_name
metrics_results, predictions, models = env_SIFT.TrainTest()
env_SIFT.evaluate_model(models[best_method_name])
metrics_results_best_methods = env_SIFT.get_metrics(models[best_method_name], "SIFT + BoW", metrics_results_best_methods)

========= 5. Classification =========
Optimization Logistic Regression
Optimization Classification Decision Tree
Optimization Random Forest
Best method name: RandomForest
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.77      0.82        86
           1       0.67      0.82      0.74        50

    accuracy                           0.79       136
   macro avg       0.78      0.79      0.78       136
weighted avg       0.80      0.79      0.79       136



In [12]:
metrics_results_best_methods

{'accuracy': {'SIFT + BoW': 0.7867647058823529},
 'f1-score': {'SIFT + BoW': 0.7793072575681272},
 'recall': {'SIFT + BoW': 0.7937209302325581},
 'precision': {'SIFT + BoW': 0.7760655737704918},
 'roc_auc': {'SIFT + BoW': 0.7937209302325581}}